In [1]:
import pandas as pd

In [2]:

# Load the dataset
data = pd.read_csv('ndtv_data_final.csv')  # Modify the path accordingly

In [3]:
data.head()

,Unnamed: 0,Name,Brand,Model,Battery capacity (mAh),Screen size (inches),Touchscreen,Resolution x,Resolution y,Processor,...,Rear camera,Front camera,Operating system,Wi-Fi,Bluetooth,GPS,Number of SIMs,3G,4G/ LTE,Price
0,0,OnePlus 7T Pro McLaren Edition,OnePlus,7T Pro McLaren Edition,4085,6.67,Yes,1440,3120,8,...,48.0,16.0,Android,Yes,Yes,Yes,2,Yes,Yes,58998
1,1,Realme X2 Pro,Realme,X2 Pro,4000,6.50,Yes,1080,2400,8,...,64.0,16.0,Android,Yes,Yes,Yes,2,Yes,Yes,27999
2,2,iPhone 11 Pro Max,Apple,iPhone 11 Pro Max,3969,6.50,Yes,1242,2688,6,...,12.0,12.0,iOS,Yes,Yes,Yes,2,Yes,Yes,106900
3,3,iPhone 11,Apple,iPhone 11,3110,6.10,Yes,828,1792,6,...,12.0,12.0,iOS,Yes,Yes,Yes,2,Yes,Yes,62900
4,4,LG G8X ThinQ,LG,G8X ThinQ,4000,6.40,Yes,1080,2340,8,...,12.0,32.0,Android,Yes,Yes,Yes,1,No,No,49990


In [4]:
data.rename(columns={'RAM (MB)': 'RAM (GB)'}, inplace=True)
data['RAM (GB)'] = data['RAM (GB)'] / 1000  # Convert MB to GB

In [5]:
# Selecting relevant columns including 'Brand' for one-hot encoding
selected_columns = ['Brand', 'Price', 'Internal storage (GB)', 'RAM (GB)', 
                    'Rear camera', 'Front camera', 'Battery capacity (mAh)','Name']
data_selected = data[selected_columns]
#Normalize the 'Brand' column to be uppercased
data_selected['Brand'] = data_selected['Brand'].str.upper()

C:\Users\sonam\AppData\Local\Temp\ipykernel_37276\4237768644.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Brand'] = data_selected['Brand'].str.upper()


In [6]:
# Specify the file name and path
file_name = 'output.xlsx'

# Export the DataFrame to an Excel file
data_selected.to_excel(file_name, index=False)

In [7]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'Brand' column
label_encoder = LabelEncoder()
data_selected['Brand'] = label_encoder.fit_transform(data_selected['Brand'])

# Display the first few rows after encoding
print(data_selected.head())

   Brand   Price  Internal storage (GB)  RAM (GB)  Rear camera  Front camera  \
0     47   58998                  256.0      12.0         48.0          16.0   
1     56   27999                   64.0       6.0         64.0          16.0   
2      3  106900                   64.0       4.0         12.0          12.0   
3      3   62900                   64.0       4.0         12.0          12.0   
4     35   49990                  128.0       6.0         12.0          32.0   

   Battery capacity (mAh)                            Name  
0                    4085  OnePlus 7T Pro McLaren Edition  
1                    4000                   Realme X2 Pro  
2                    3969               iPhone 11 Pro Max  
3                    3110                       iPhone 11  
4                    4000                    LG G8X ThinQ  


C:\Users\sonam\AppData\Local\Temp\ipykernel_37276\3982285942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Brand'] = label_encoder.fit_transform(data_selected['Brand'])


In [8]:
from sklearn.preprocessing import StandardScaler

# Split the data into features and target
X = data_selected.drop('Name', axis=1,errors="coerce")
y = data_selected['Name']

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Display the first few rows of the standardized features
print(pd.DataFrame(X_scaled, columns=X.columns).head())


      Brand     Price  Internal storage (GB)  RAM (GB)  Rear camera  \
0  0.425726  3.431332               6.100856  5.716470     4.016727   
1  0.887139  1.193524               0.902766  2.110328     5.805427   
2 -1.830074  6.889361               0.902766  0.908280    -0.007848   
3 -1.830074  3.713016               0.902766  0.908280    -0.007848   
4 -0.189492  2.781047               2.635462  2.110328    -0.007848   

   Front camera  Battery capacity (mAh)  
0      1.424097                1.313010  
1      1.424097                1.215666  
2      0.788484                1.180164  
3      0.788484                0.196418  
4      3.966552                1.215666  


In [9]:
# Example input features
input_features = {
    'Brand': 'LG',
    'Price': 100000,
    'Internal storage (GB)': 128,
    'RAM (GB)': 8,
    'Rear camera': 48,
    'Front camera': 16,
    'Battery capacity (mAh)': 4000
}

In [10]:
from sklearn.neighbors import NearestNeighbors

# Fit the k-NN model for nearest neighbors search
knn = NearestNeighbors(n_neighbors=5)
knn.fit(X_scaled)


NearestNeighbors()

In [11]:
def recommend_phone(input_features):
 
    # Convert input features to DataFrame
    input_df = pd.DataFrame([input_features])
    
    # Encode the 'Brand' column
    input_df['Brand'] = label_encoder.transform(input_df['Brand'])
    
    # Standardize the input features
    input_scaled = scaler.transform(input_df)
    
    # Find the nearest neighbors
    distances, indices = knn.kneighbors(input_scaled)
    
    # Get the recommended phone names
    recommended_phones = y.iloc[indices[0]].values
    
    return recommended_phones



# Recommend phones
print(recommend_phone(input_features))


['Huawei Mate 30 Pro' 'Huawei Mate 20 Pro' 'Nubia Red Magic 3S'
 'OnePlus 7 Pro' 'Google Pixel 4 XL']


# 

In [12]:
from sklearn.cluster import KMeans

# Fit the K-Means model
kmeans = KMeans(n_clusters=3, random_state=40)
kmeans.fit(X_scaled)

C:\Users\sonam\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\sonam\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


KMeans(n_clusters=3, random_state=40)

In [13]:

# Implement the Recommendation Function for K-Means
def recommend_phone_kmeans(input_features):
    """
    Recommend phones based on input features using K-Means.
    input_features: dict containing values for the selected columns
    """
    # Convert input features to DataFrame
    input_df = pd.DataFrame([input_features])
    
    # Encode the 'Brand' column
    input_df['Brand'] = label_encoder.transform(input_df['Brand'])
    
    # Standardize the input features
    input_scaled = scaler.transform(input_df)
    
    # Predict the cluster for the input features
    cluster = kmeans.predict(input_scaled)
    
    # Get the indices of phones in the same cluster
    indices = (kmeans.labels_ == cluster[0]).nonzero()[0]
    
    # Get the recommended phone names
    recommended_phones = y.iloc[indices].values
    
    return recommended_phones

# Recommend phones using K-Means
print(recommend_phone_kmeans(input_features))

['OnePlus 7T Pro McLaren Edition' 'Realme X2 Pro' 'iPhone 11 Pro Max'
 'LG G8X ThinQ' 'OnePlus 7T' 'OnePlus 7T Pro' 'Samsung Galaxy Note 10+'
 'Asus ROG Phone 2' 'Xiaomi Redmi K20 Pro' 'Realme X' 'Xiaomi Redmi K20'
 'OnePlus 7 Pro' 'Oppo Reno 10x Zoom' 'Huawei P30 Pro' 'Redmi Note 7 Pro'
 'Huawei Mate 20 Pro' 'LG V40 ThinQ' 'OnePlus 6T' 'Asus ROG Phone'
 'Samsung Galaxy Note 9' 'LG G7+ ThinQ' 'Huawei P20 Pro' 'HTC U11+'
 'HTC U11' 'Asus ZenFone 3 Deluxe (ZS570KL)' 'Poco X2'
 'Samsung Galaxy S10 Lite' 'Samsung Galaxy Note 10 Lite' 'Vivo V17'
 'Realme 5s' 'Realme X2' 'Vivo V17 Pro' 'Nubia Red Magic 3S'
 'Samsung Galaxy M30s' 'Vivo Z1x' 'Oppo Reno 2Z' 'Realme XT'
 'Redmi Note 8 Pro' 'Nokia 7.2' 'Samsung Galaxy A50s' 'Realme 5 Pro'
 'Vivo Z1 Pro' 'Xiaomi Mi A3' 'Asus 6Z' 'Samsung Galaxy M40'
 'Motorola One Vision' 'Nubia Red Magic 3' 'Samsung Galaxy A70'
 'Black Shark 2' 'OnePlus 7' 'Vivo V15 Pro' 'Honor View 20'
 'Samsung Galaxy A9 (2018)' 'Oppo R17 Pro' 'Vivo Nex' 'Oppo Find X'
 'Honor 1

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [15]:
# Fit the Decision Tree model
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

In [16]:
# Evaluate the Decision Tree model
y_pred = dt.predict(X_test)
print(f"Decision Tree Accuracy: {accuracy_score(y_test, y_pred)}")


Decision Tree Accuracy: 0.0


In [17]:
# Implement the Recommendation Function for Decision Tree
def recommend_phone_dt(input_features):
    """
    Recommend phones based on input features using Decision Tree.
    input_features: dict containing values for the selected columns
    """
    # Convert input features to DataFrame
    input_df = pd.DataFrame([input_features])
    
    # Encode the 'Brand' column
    input_df['Brand'] = label_encoder.transform(input_df['Brand'])
    
    # Standardize the input features
    input_scaled = scaler.transform(input_df)
    
    # Predict the phone name
    recommended_phone = dt.predict(input_scaled)
    
    return recommended_phone

# Recommend phones using Decision Tree
print(recommend_phone_dt(input_features))

['Oppo Reno 2F']


In [18]:
from sklearn.ensemble import RandomForestClassifier

# Fit the Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluate the Random Forest model
y_pred_rf = rf.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf)}")


Random Forest Accuracy: 0.0


In [19]:
# Implement the Recommendation Function for Random Forest
def recommend_phone_rf(input_features):
    """
    Recommend phones based on input features using Random Forest.
    input_features: dict containing values for the selected columns
    """
    # Convert input features to DataFrame
    input_df = pd.DataFrame([input_features])
    
    # Encode the 'Brand' column
    input_df['Brand'] = label_encoder.transform(input_df['Brand'])
    
    # Standardize the input features
    input_scaled = scaler.transform(input_df)
    
    # Predict the phone name
    recommended_phone = rf.predict(input_scaled)
    
    return recommended_phone

# Recommend phones using Random Forest
print(recommend_phone_rf(input_features))

['Huawei Mate 30 Pro']
